In [1]:
!pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.4 MB 2.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.4 MB 2.5 MB/s eta 0:00:04
   ------ --------------------------------- 1.6/9.4 MB 2.4 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.4 MB 2.1 MB/s eta 0:00:04
   ----------- ---------------------------- 2.6/9.4 MB 2.1 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.4 MB 2.2 MB/s eta 0:00:03
   --------------- ------------------------ 3.7/9.4 MB 2.2 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.4 MB 2.2 MB/s eta 0:00:03
   ------------------ --------------------- 4.5/9.4 MB 2.2 MB/s eta 0:00:03
   --------------------- ------------------ 5.0/9.4 MB 2.2 MB/s eta 0:00:03
   ----------------------- ---------------- 5.5/9.4 MB 2.3 MB/s eta 0:00:02
   -----------------------

In [2]:
!pip install undetected-chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47184 sha256=639d68c6868c6bf8c4d8acb41b2e305bfe4883719c1d8f9b783e07abbfbf4bde
  Stored in directory: c:\users\evi_r\appdata\local\pip\cache\wheels\c4\f1\aa\9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected-chromedriver


In [6]:
import undetected_chromedriver as uc

driver = uc.Chrome(headless=True, use_subprocess=False)
driver.get('https://nowsecure.nl')

In [7]:
!pip install selenium-stealth

In [29]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

datei = "tandfonline_journals.xlsx"
ziel_datei = "tandfonline_journals_seleniumAll.xlsx"
delay = 5

df = pd.read_excel(eingabe_datei)

def create_driver():
    options = Options()
    options.add:argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def extract_metrics_with_selenium(url):
    driver = create_driver()
    try:
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        container = soup.find("div", class_="journalMetrics")
        if not container:
            return {"Fehler": "journalMetrics-Container nicht gefunden"}

        metrics = {}
        
        usage_div = container.find("div", class_="usage")
        if usage_div:
            li = usage_div.find("li")
            if li:
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)
                    key = li.get_text(strip=True).replace(value, "").strip()
                    metrics[key] = value
                else:
                    metrics["Usage"] = li.get_text(strip=True)

        citation_div = container.find("div", class_="citation-metrics")
        if citation_div:
            for li in citation_div.find_all("li"):
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)
                    key = li.get_text(strip=True).replace(value, "").strip()
                    metrics[key] = value
                else:
                    metrics["Citation"] = li.get_text(strip=True)

        speed_div = container.find("div", class_="speed")
        if speed_div:
            for li in speed_div.find_all("li"):
                strong_tag = li.find("strong")
                if strong_tag:
                    value = strong_tag.get_text(strip=True)
                    key = li.get_text(strip=True).replace(value, "").strip()
                    metrics[key] = value
                else:
                    metrics["Speed/Acceptance"] = li.get_text(strip=True)
               
        return metrics
        
    except Exception as e:
        return {"Fehler": str(e)}

    finally:
        driver.quit()

alle_metrics = []
for i, row in df.iterrows():
    url = row["URL"]
    print(f"[{i+1}/{len(df)}]Scraping: {url}")
    metrics = extract_metrics_with_selenium(url)
    alle_metrics.append(metrics)
    print(f"Fertig mit URL {i+1} warte {delay} Sekunden...")
    time.sleep(delay)

metrics_df = pd.DataFrame(alle_metrics)
df_out = pd.concat([df, metrics_df], axis=1)

df_out.to_excel(ziel_datei, index=False)
print(f"Fertig! Datei gespeichert unter: {ziel_datei}")


[1/6]Scraping: https://www.tandfonline.com/journals/rdsp20/about-this-journal#journal-metrics
Fertig mit URL 1 warte 5 Sekunden...
[2/6]Scraping: https://www.tandfonline.com/journals/ceps20/about-this-journal#journal-metrics
Fertig mit URL 2 warte 5 Sekunden...
[3/6]Scraping: https://www.tandfonline.com/journals/cjoe20/about-this-journal#journal-metrics
Fertig mit URL 3 warte 5 Sekunden...
[4/6]Scraping: https://www.tandfonline.com/journals/nens20/about-this-journal#journal-metrics
Fertig mit URL 4 warte 5 Sekunden...
[5/6]Scraping: https://www.tandfonline.com/journals/rptp20/about-this-journal#journal-metrics
Fertig mit URL 5 warte 5 Sekunden...
[6/6]Scraping: https://www.tandfonline.com/journals/tsus20/about-this-journal#journal-metrics
Fertig mit URL 6 warte 5 Sekunden...
Fertig! Datei gespeichert unter: tandfonline_journals_seleniumAll.xlsx
